# Detecting Hidden Discrimination in the Adult Dataset
This notebook demonstrates how to use the discrimination detection framework to identify and mitigate hidden discrimination in the Adult dataset.
# Setup
First, let's import all the necessary modules:

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Add the parent directory to the path to import our modules
sys.path.append('..')

# Import our modules
from src.preprocessing import load_adult_dataset, preprocess_adult_dataset
from src.clustering import MultiClusteringAlgorithm
from src.cmi import calculate_cmi, calculate_cmi_per_cluster, hierarchical_cmi_decomposition, interaction_information
from src.validation import permutation_test, bootstrap_ci, plot_permutation_test, plot_bootstrap_distribution
from src.mitigation import reweighting, FairnessRegularizedModel, subgroup_calibration, evaluate_mitigation, plot_mitigation_comparison

# 1. Load and Preprocess Data
Let's load the Adult dataset and preprocess it as described in the paper:


In [ ]:
# Download Adult dataset if not already available
import urllib.request
import os

data_dir = '../data'
adult_data_path = os.path.join(data_dir, 'adult.data')

if not os.path.exists(adult_data_path):
    print("Downloading Adult dataset...")
    os.makedirs(data_dir, exist_ok=True)
    urllib.request.urlretrieve(
        'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
        adult_data_path
    )
    print("Download complete.")

# Load the dataset
adult_data = load_adult_dataset(adult_data_path)

# Display basic info
print(f"Dataset shape: {adult_data.shape}")
print(adult_data.head())

# Preprocess the dataset
processed_data, sensitive_columns, nonsensitive_columns, outcome_column = preprocess_adult_dataset(adult_data)

print(f"Processed dataset shape: {processed_data.shape}")
print(f"Sensitive columns: {sensitive_columns}")
print(f"Outcome column: {outcome_column}")

Let's examine the distribution of the sensitive attributes and outcome:

In [ ]:
# Look at distributions of sensitive attributes
fig, axes = plt.subplots(1, len(sensitive_columns) + 1, figsize=(15, 5))

for i, col in enumerate(sensitive_columns):
    counts = processed_data[col].value_counts().sort_index()
    axes[i].bar(counts.index, counts.values)
    axes[i].set_title(f'Distribution of {col}')
    axes[i].set_ylabel('Count')
    axes[i].grid(alpha=0.3)
    

# Also show outcome distribution
counts = processed_data[outcome_column].value_counts().sort_index()
axes[-1].bar(counts.index, counts.values)
axes[-1].set_title(f'Distribution of {outcome_column}')
axes[-1].set_ylabel('Count')
axes[-1].set_xticks([0, 1])
axes[-1].set_xticklabels(['<=50K', '>50K'])
axes[-1].grid(alpha=0.3)

    

plt.tight_layout()
plt.show()

Now, let's look at the relationship between sensitive attributes and outcome:

In [ ]:

# Create a figure to show relationship between sensitive attributes and outcome
fig, axes = plt.subplots(1, len(sensitive_columns), figsize=(15, 5))

for i, col in enumerate(sensitive_columns):
    # Calculate outcome rate for each value of the sensitive attribute
    group_outcomes = processed_data.groupby(col)[outcome_column].mean()
    
    axes[i].bar(group_outcomes.index, group_outcomes.values)
    axes[i].set_title(f'Outcome Rate by {col}')
    axes[i].set_ylabel('Probability of >50K')
    axes[i].set_ylim(0, 1)
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 2. Split Data and Perform Clustering
Let's split the data into training and testing sets, then perform clustering on the non-sensitive attributes:

In [ ]:
# Split data into training and testing sets
X = processed_data[nonsensitive_columns]
y = processed_data[outcome_column]
sensitive = processed_data[sensitive_columns]

X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(
    X, y, sensitive, test_size=0.3, random_state=42
)

# Reconstruct DataFrames
train_data = pd.concat([X_train, sensitive_train, y_train], axis=1)
test_data = pd.concat([X_test, sensitive_test, y_test], axis=1)

print(f"Training set: {len(train_data)} samples")
print(f"Testing set: {len(test_data)} samples")

Now, let's try different clustering algorithms and find the optimal number of clusters:


In [ ]:
# Initialize the clustering algorithm
clustering = MultiClusteringAlgorithm()

# Try finding optimal number of clusters using the Gap statistic
# Note: This can be time-consuming, so we'll use a subset of the data
sample_size = min(10000, len(X_train))
X_sample = X_train.sample(sample_size, random_state=42).values

# Find optimal k (using k-means for speed)
optimal_k, gap_values = clustering.find_optimal_k(
    X_sample, 
    algorithm='kmeans', 
    k_range=range(2, 11),
    random_state=42
)

print(f"Optimal number of clusters according to Gap statistic: {optimal_k}")

# Plot the Gap statistic
clustering.plot_gap_statistic(range(2, 11), gap_values)

Now that we have the optimal number of clusters, let's compare different clustering algorithms:

In [ ]:
# Try different clustering algorithms with the optimal k
algorithms = ['kmeans', 'gmm', 'spectral', 'ensemble']
cluster_results = {}
cmi_values = {}

for algorithm in algorithms:
    print(f"\nTrying {algorithm} clustering...")
    try:
        if algorithm == 'ensemble':
            clusters = clustering.ensemble_clustering(X_train.values, n_clusters=optimal_k, random_state=42)
        else:
            clusters = clustering.fit(X_train.values, algorithm=algorithm, n_clusters=optimal_k, random_state=42)
        
        # Evaluate clustering quality
        metrics = clustering.evaluate_clusters(X_train.values, clusters)
        print(f"Silhouette score: {metrics['silhouette_score']:.3f}")
        print(f"Davies-Bouldin index: {metrics['davies_bouldin_index']:.3f}")
        
        # Calculate CMI
        cmi = calculate_cmi(
            train_data, 
            clusters, 
            sensitive_columns, 
            outcome_column, 
            nonsensitive_columns
        )
        print(f"CMI: {cmi:.4f}")
        
        # Store results
        cluster_results[algorithm] = clusters
        cmi_values[algorithm] = cmi
    except Exception as e:
        print(f"Error with {algorithm}: {e}")

Let's visualize the clusters from the best algorithm

In [ ]:
# Find the algorithm with the highest CMI (best at revealing discrimination)
best_algorithm = max(cmi_values, key=cmi_values.get)
best_clusters = cluster_results[best_algorithm]

print(f"Best algorithm: {best_algorithm} with CMI = {cmi_values[best_algorithm]:.4f}")

# Visualize the clusters using PCA
clustering.plot_clusters(X_train.values, best_clusters)

# Also visualize with sensitive attributes coloring
if 'sex' in sensitive_columns:
    clustering.plot_clusters(
        X_train.values, 
        best_clusters, 
        sensitive_attr=sensitive_train['sex'].values
    )

if 'race' in sensitive_columns:
    clustering.plot_clusters(
        X_train.values, 
        best_clusters, 
        sensitive_attr=sensitive_train['race'].values
    )

# 3. Calculate CMI and Identify High-Discrimination Clusters
Now, let's calculate the CMI for each cluster to identify ones with high discrimination:


In [ ]:
# Calculate CMI per cluster
print("Calculating CMI for each cluster...")
cmi_per_cluster = calculate_cmi_per_cluster(
    train_data,
    best_clusters,
    sensitive_columns,
    outcome_column,
    nonsensitive_columns
)

# Add clusters to training data for further analysis
train_data_with_clusters = train_data.copy()
train_data_with_clusters['cluster'] = best_clusters

# Plot CMI by cluster
plt.figure(figsize=(10, 6))
clusters = list(cmi_per_cluster.keys())
cmi_values_list = list(cmi_per_cluster.values())

# Sort by CMI value
sorted_indices = np.argsort(cmi_values_list)[::-1]
sorted_clusters = [clusters[i] for i in sorted_indices]
sorted_values = [cmi_values_list[i] for i in sorted_indices]

plt.bar(sorted_clusters, sorted_values)
plt.xlabel('Cluster')
plt.ylabel('CMI')
plt.title('Conditional Mutual Information by Cluster')
plt.xticks(sorted_clusters)
plt.grid(True, alpha=0.3)
plt.show()

# Identify high-discrimination clusters
threshold = np.mean(list(cmi_per_cluster.values())) + 0.5 * np.std(list(cmi_per_cluster.values()))
high_discrim_clusters = [c for c, v in cmi_per_cluster.items() if v > threshold]

print("\nCMI per cluster:")
for cluster_id, cluster_cmi in sorted(cmi_per_cluster.items(), key=lambda x: x[1], reverse=True):
    print(f"  Cluster {cluster_id}: {cluster_cmi:.4f}" + 
          (" (high discrimination)" if cluster_id in high_discrim_clusters else ""))

In [ ]:
# Plot CMI by cluster
plt.figure(figsize=(10, 6))
clusters = list(cmi_per_cluster.keys())
cmi_values_list = list(cmi_per_cluster.values())

# Sort by CMI value
sorted_indices = np.argsort(cmi_values_list)[::-1]
sorted_clusters = [clusters[i] for i in sorted_indices]
sorted_values = [cmi_values_list[i] for i in sorted_indices]

# Define discrimination thresholds
high_threshold = 0.1
medium_threshold = 0.05

# Color bars based on discrimination level
colors = ['red' if val > high_threshold else 'orange' if val > medium_threshold else 'blue' for val in sorted_values]

# Create bar chart with colors
bars = plt.bar(sorted_clusters, sorted_values, color=colors)

# Add threshold lines
plt.axhline(y=high_threshold, color='red', linestyle='--', label='High discrimination (0.1)')
plt.axhline(y=medium_threshold, color='orange', linestyle='--', label='Medium discrimination (0.05)')

plt.xlabel('Cluster')
plt.ylabel('CMI')
plt.title('Discrimination Level by Cluster')
plt.xticks(sorted_clusters)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('discrimination_by_cluster.png', dpi=300)
plt.show()

# Identify high-discrimination clusters
print("\nCMI per cluster:")
for cluster_id, cluster_cmi in sorted(cmi_per_cluster.items(), key=lambda x: x[1], reverse=True):
    discrimination_level = "high discrimination" if cluster_cmi > high_threshold else \
                          "medium discrimination" if cluster_cmi > medium_threshold else \
                          "low discrimination"
    print(f"  Cluster {cluster_id}: {cluster_cmi:.4f} ({discrimination_level})")

Let's analyze the characteristics of high-discrimination clusters:

In [ ]:
for cluster_id in high_discrim_clusters:
    print(f"\nCluster {cluster_id} characteristics:")
    cluster_data = train_data_with_clusters[train_data_with_clusters['cluster'] == cluster_id]
    
    # Outcome rate
    outcome_rate = cluster_data[outcome_column].mean()
    print(f"  Outcome rate: {outcome_rate:.2f}")
    print(f"  Size: {len(cluster_data)} samples")
    
    # Distribution by sensitive attributes
    for col in sensitive_columns:
        print(f"\n  Distribution by {col}:")
        value_counts = cluster_data[col].value_counts(normalize=True)
        
        for value, proportion in value_counts.items():
            # Get outcome rate for this subgroup
            subgroup = cluster_data[cluster_data[col] == value]
            subgroup_outcome_rate = subgroup[outcome_column].mean()
            
            print(f"    {col}={value}: {proportion:.2f} of cluster, outcome rate: {subgroup_outcome_rate:.2f}")

In [ ]:
for cluster_id in high_discrim_clusters:
    print(f"\n{'='*50}")
    print(f"CLUSTER {cluster_id} DETAILED ANALYSIS")
    print(f"{'='*50}")
    
    # Get data for this cluster
    cluster_data = train_data_with_clusters[train_data_with_clusters['cluster'] == cluster_id]
    
    # Basic cluster statistics
    outcome_rate = cluster_data[outcome_column].mean()
    cluster_size = len(cluster_data)
    cluster_percentage = (cluster_size / len(train_data_with_clusters)) * 100
    
    print(f"Size: {cluster_size} samples ({cluster_percentage:.1f}% of total data)")
    print(f"Overall outcome rate: {outcome_rate:.2f}")
    
    # CMI value for this cluster
    cluster_cmi = cmi_per_cluster.get(cluster_id, "Not calculated")
    print(f"Discrimination level (CMI): {cluster_cmi:.4f}")
    
    # Analyze individual sensitive attributes
    print("\nSENSITIVE ATTRIBUTE ANALYSIS:")
    for col in sensitive_columns:
        print(f"\n  Distribution by {col}:")
        value_counts = cluster_data[col].value_counts()
        total = value_counts.sum()
        
        # Calculate divergence from overall outcome rate
        max_divergence = 0
        max_divergence_group = None
        
        # Create a formatted table for output
        print(f"    {'Value':<15} {'Count':<8} {'Proportion':<12} {'Outcome Rate':<15} {'Divergence':<10}")
        print(f"    {'-'*60}")
        
        for value, count in value_counts.items():
            proportion = count / total
            subgroup = cluster_data[cluster_data[col] == value]
            subgroup_outcome_rate = subgroup[outcome_column].mean()
            divergence = abs(subgroup_outcome_rate - outcome_rate)
            
            # Track maximum divergence
            if divergence > max_divergence:
                max_divergence = divergence
                max_divergence_group = value
                
            # Format and print the row
            print(f"    {str(value):<15} {count:<8} {proportion:.2f} ({proportion*100:.1f}%) {subgroup_outcome_rate:.2f} {divergence:+.2f}")
        
        print(f"\n  → Highest divergence in {col}: {max_divergence_group} ({max_divergence:+.2f})")
    
    # Intersectional analysis
    if len(sensitive_columns) > 1:
        print("\nINTERSECTIONAL ANALYSIS:")
        # Create intersection column
        cluster_data['intersection'] = cluster_data[sensitive_columns].apply(
            lambda row: '_'.join(str(row[col]) for col in sensitive_columns), axis=1
        )
        
        # Get intersectional counts and rates
        intersection_counts = cluster_data['intersection'].value_counts()
        
        # Only show intersections with at least 20 samples (for statistical reliability)
        threshold = 20
        valid_intersections = intersection_counts[intersection_counts >= threshold]
        
        if len(valid_intersections) > 0:
            print(f"\n  Showing intersections with at least {threshold} samples:")
            print(f"    {'Intersection':<25} {'Count':<8} {'Proportion':<12} {'Outcome Rate':<15} {'Divergence':<10}")
            print(f"    {'-'*70}")
            
            for intersection, count in valid_intersections.items():
                proportion = count / total
                subgroup = cluster_data[cluster_data['intersection'] == intersection]
                subgroup_outcome_rate = subgroup[outcome_column].mean()
                divergence = subgroup_outcome_rate - outcome_rate
                
                print(f"    {intersection:<25} {count:<8} {proportion:.2f} ({proportion*100:.1f}%) {subgroup_outcome_rate:.2f} {divergence:+.2f}")
        else:
            print(f"  No intersections with at least {threshold} samples found.")
    
    # Feature importance for this cluster
    print("\nKEY FEATURES CHARACTERIZING THIS CLUSTER:")
    # Get all non-sensitive columns except the outcome and cluster columns
    feature_columns = [col for col in cluster_data.columns 
                      if col not in sensitive_columns + [outcome_column, 'cluster', 'intersection']]
    
    # Calculate the mean value of each feature in this cluster vs. overall
    feature_comparison = []
    for col in feature_columns:
        try:
            # Only analyze numerical columns
            if pd.api.types.is_numeric_dtype(cluster_data[col]):
                cluster_mean = cluster_data[col].mean()
                overall_mean = train_data_with_clusters[col].mean()
                std_dev = train_data_with_clusters[col].std()
                
                # Calculate z-score to measure how distinctive this feature is
                if std_dev > 0:
                    z_score = (cluster_mean - overall_mean) / std_dev
                    feature_comparison.append((col, cluster_mean, overall_mean, z_score))
        except:
            # Skip columns that cause errors
            continue
    
    # Sort by absolute z-score and display top features
    top_n = min(10, len(feature_comparison))
    feature_comparison.sort(key=lambda x: abs(x[3]), reverse=True)
    
    if feature_comparison:
        print(f"  Top {top_n} distinctive features:")
        print(f"    {'Feature':<20} {'Cluster Mean':<15} {'Overall Mean':<15} {'Z-Score':<10}")
        print(f"    {'-'*60}")
        
        for i in range(top_n):
            col, c_mean, o_mean, z = feature_comparison[i]
            print(f"    {col:<20} {c_mean:.2f} {o_mean:.2f} {z:+.2f}")
    else:
        print("  No numerical features available for analysis.")
    
    print(f"\n{'='*50}\n")

In [ ]:

# Set up the visualization style
sns.set(style="whitegrid")
plt.rcParams.update({'font.size': 11})

# Create a figure for comparison of all high discrimination clusters
fig_overview = plt.figure(figsize=(10, 6))
ax_overview = fig_overview.add_subplot(111)

# Prepare data for overview comparison
cluster_summary = []

# Loop through each high discrimination cluster
for cluster_id in high_discrim_clusters:
    cluster_data = train_data_with_clusters[train_data_with_clusters['cluster'] == cluster_id]
    
    # Outcome rate
    outcome_rate = cluster_data[outcome_column].mean()
    print(f"\nCluster {cluster_id} characteristics:")
    print(f"  Outcome rate: {outcome_rate:.2f}")
    print(f"  Size: {len(cluster_data)} samples")
    
    # Store summary data for overview plot
    cluster_summary.append({
        'cluster_id': cluster_id,
        'size': len(cluster_data),
        'outcome_rate': outcome_rate
    })
    
    # Create a figure for this cluster's detailed analysis
    fig, axes = plt.subplots(len(sensitive_columns), 2, figsize=(15, 4 * len(sensitive_columns)))
    fig.suptitle(f'Cluster {cluster_id} Characteristics (Size: {len(cluster_data)} samples, Overall outcome rate: {outcome_rate:.2f})', 
                fontsize=16)
    
    # Analyze each sensitive attribute
    for idx, col in enumerate(sensitive_columns):
        print(f"\n  Distribution by {col}:")
        
        # Get value counts and prepare visualization data
        value_counts = cluster_data[col].value_counts(normalize=True)
        subgroup_stats = []
        
        for value, proportion in value_counts.items():
            # Get outcome rate for this subgroup
            subgroup = cluster_data[cluster_data[col] == value]
            subgroup_outcome_rate = subgroup[outcome_column].mean()
            
            print(f"    {col}={value}: {proportion:.2f} of cluster, outcome rate: {subgroup_outcome_rate:.2f}")
            
            # Store for plotting
            subgroup_stats.append({
                'value': str(value),
                'proportion': proportion,
                'outcome_rate': subgroup_outcome_rate,
                'size': len(subgroup)
            })
        
        # Convert to DataFrame for easier plotting
        subgroup_df = pd.DataFrame(subgroup_stats)
        
        # Left plot: Distribution within cluster by sensitive attribute
        if len(sensitive_columns) == 1:
            ax1 = axes[0]
            ax2 = axes[1]
        else:
            ax1 = axes[idx, 0]
            ax2 = axes[idx, 1]
            
        # Create horizontal bar chart for distribution
        bars = ax1.barh(subgroup_df['value'], subgroup_df['proportion'], color='skyblue')
        ax1.set_title(f'Distribution by {col}')
        ax1.set_xlabel('Proportion within cluster')
        ax1.set_xlim(0, 1)
        
        # Add proportion labels
        for bar in bars:
            width = bar.get_width()
            ax1.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
                    f'{width:.2f}', va='center')
        
        # Right plot: Outcome rates by subgroup
        bars = ax2.barh(subgroup_df['value'], subgroup_df['outcome_rate'], color='lightcoral')
        ax2.set_title(f'Outcome Rates by {col}')
        ax2.set_xlabel(f'{outcome_column} Rate')
        ax2.set_xlim(0, 1)
        
        # Add outcome rate labels and sample size
        for i, bar in enumerate(bars):
            width = bar.get_width()
            ax2.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
                    f'{width:.2f} (n={subgroup_df.iloc[i]["size"]})', va='center')
        
        # Add a reference line for the overall cluster outcome rate
        ax2.axvline(x=outcome_rate, color='red', linestyle='--', 
                   label=f'Cluster average: {outcome_rate:.2f}')
        ax2.legend()
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to make room for the title
    
    # Save the figure
    plt.savefig(f'cluster_{cluster_id}_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

# Create overview plot comparing all high discrimination clusters
cluster_df = pd.DataFrame(cluster_summary)

# Sort by outcome rate for better visualization
cluster_df = cluster_df.sort_values('outcome_rate', ascending=False)

# Plot comparisons
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('High Discrimination Clusters Comparison', fontsize=16)

# Left: Cluster sizes
size_bars = ax1.bar([f'Cluster {cid}' for cid in cluster_df['cluster_id']], 
                    cluster_df['size'], color='lightblue')
ax1.set_title('Cluster Sizes')
ax1.set_xlabel('Cluster')
ax1.set_ylabel('Number of Samples')

# Add size labels
for bar in size_bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2, height + 5, 
            f'{int(height)}', ha='center')

# Right: Outcome rates
rate_bars = ax2.bar([f'Cluster {cid}' for cid in cluster_df['cluster_id']], 
                   cluster_df['outcome_rate'], color='salmon')
ax2.set_title(f'{outcome_column} Rates')
ax2.set_xlabel('Cluster')
ax2.set_ylabel('Rate')
ax2.set_ylim(0, 1)

# Add rate labels
for bar in rate_bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2, height + 0.01, 
            f'{height:.2f}', ha='center')

# Overall dataset average for reference
overall_rate = train_data_with_clusters[outcome_column].mean()
ax2.axhline(y=overall_rate, color='red', linestyle='--', 
           label=f'Dataset average: {overall_rate:.2f}')
ax2.legend()

plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save the overview comparison figure
plt.savefig('high_discrimination_clusters_comparison.png', dpi=600, bbox_inches='tight')
plt.show()

# 4. Statistical Validation
Let's validate our findings for one of the high-discrimination clusters:


In [ ]:
# Pick the cluster with the highest CMI
if high_discrim_clusters:
    target_cluster = high_discrim_clusters[0]
    
    # Extract data for this cluster
    cluster_data = train_data_with_clusters[train_data_with_clusters['cluster'] == target_cluster]
    cluster_size = len(cluster_data)
    
    print(f"Validating discrimination in cluster {target_cluster} (size: {cluster_size})...")
    
    # Set all samples to same cluster (since we're analyzing within a single cluster)
    cluster_assignments = np.zeros(cluster_size)
    
    # Perform permutation test
    print("\nPerforming permutation test...")
    perm_results = permutation_test(
        cluster_data,
        cluster_assignments,
        sensitive_columns,
        outcome_column,
        nonsensitive_columns,
        num_permutations=100  # Use more (e.g., 1000) for a real analysis
    )
    
    # Plot permutation test results
    plot_permutation_test(perm_results)
   
  

    
    # Bootstrap confidence interval
    print("\nCalculating bootstrap confidence interval...")
    bootstrap_results = bootstrap_ci(
        cluster_data,
        cluster_assignments,
        sensitive_columns,
        outcome_column,
        nonsensitive_columns,
        num_bootstraps=1000  # Use more (e.g., 1000) for a real analysis
    )
    # Save bootstrap CI plot
    plot_bootstrap_distribution(bootstrap_results)
    fig = plt.gcf()
    fig.savefig(f'cluster_{target_cluster}_bootstrap_distribution.png', dpi=300, bbox_inches='tight')
    plt.close(fig)

    

    # Plot bootstrap distribution
    plot_bootstrap_distribution(bootstrap_results)
else:
    print("No high-discrimination clusters identified.")

In [ ]:
# Pick the cluster with the highest CMI
if high_discrim_clusters:
    target_cluster = high_discrim_clusters[0]
    
    # Extract data for this cluster
    cluster_data = train_data_with_clusters[train_data_with_clusters['cluster'] == target_cluster]
    cluster_size = len(cluster_data)
    
    print(f"Validating discrimination in cluster {target_cluster} (size: {cluster_size})...")
    
    # Set all samples to same cluster (since we're analyzing within a single cluster)
    cluster_assignments = np.zeros(cluster_size)
    
    # Perform permutation test
    print("\nPerforming permutation test...")
    perm_results = permutation_test(
        cluster_data,
        cluster_assignments,
        sensitive_columns,
        outcome_column,
        nonsensitive_columns,
        num_permutations=100  # Use more (e.g., 1000) for a real analysis
    )
    
    # Plot permutation test results and save
    # Assuming plot_permutation_test doesn't return a figure but creates one
    plot_permutation_test(perm_results)
    plt.savefig(f'cluster_{target_cluster}_permutation_test.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Bootstrap confidence interval
    print("\nCalculating bootstrap confidence interval...")
    bootstrap_results = bootstrap_ci(
        cluster_data,
        cluster_assignments,
        sensitive_columns,
        outcome_column,
        nonsensitive_columns,
        num_bootstraps=1000  # Use more (e.g., 1000) for a real analysis
    )
    
    # Plot bootstrap distribution and save
    # Assuming plot_bootstrap_distribution doesn't return a figure but creates one
    plot_bootstrap_distribution(bootstrap_results)
    plt.savefig(f'cluster_{target_cluster}_bootstrap_ci.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create and save a combined visualization showing key metrics
    plt.figure(figsize=(12, 8))
    
    # Extract p-value and observed CMI from permutation results
    p_value = perm_results['p_value']
    observed_cmi = perm_results['observed_cmi']
    
    # Handle different possible formats of bootstrap_results
    # Check the structure of bootstrap_results
    if isinstance(bootstrap_results, dict) and 'ci' in bootstrap_results:
        lower_ci, upper_ci = bootstrap_results['ci']
    elif isinstance(bootstrap_results, dict) and 'lower_ci' in bootstrap_results and 'upper_ci' in bootstrap_results:
        lower_ci = bootstrap_results['lower_ci']
        upper_ci = bootstrap_results['upper_ci']
    elif isinstance(bootstrap_results, tuple) and len(bootstrap_results) == 2:
        lower_ci, upper_ci = bootstrap_results
    else:
        # If we can't determine the CI, use observed value with a default margin
        print("Warning: Could not extract CI from bootstrap results. Using default CI.")
        lower_ci = observed_cmi * 0.9
        upper_ci = observed_cmi * 1.1
    
    # Create summary visualization
    plt.subplot(2, 2, 1)
    plt.bar(['Observed CMI'], [observed_cmi], color='blue')
    plt.title(f'Observed CMI: {observed_cmi:.4f}')
    plt.ylabel('CMI Value')
    
    plt.subplot(2, 2, 2)
    plt.bar(['P-value'], [p_value], color='red' if p_value < 0.05 else 'green')
    plt.title(f'Permutation Test P-value: {p_value:.4f}')
    plt.axhline(y=0.05, color='r', linestyle='--', label='Significance threshold')
    plt.ylim(0, 1)
    
    plt.subplot(2, 2, 3)
    plt.errorbar(['CMI 95% CI'], [observed_cmi], yerr=[[observed_cmi-lower_ci], [upper_ci-observed_cmi]], 
                fmt='o', capsize=10, color='purple')
    plt.title(f'Bootstrap 95% CI: [{lower_ci:.4f}, {upper_ci:.4f}]')
    
    plt.subplot(2, 2, 4)
    # Show cluster info
    cluster_info = (f"Cluster {target_cluster}\n"
                   f"Size: {cluster_size} samples\n"
                   f"Outcome rate: {cluster_data[outcome_column].mean():.2f}\n"
                   f"Significant: {'Yes' if p_value < 0.05 else 'No'}")
    plt.text(0.5, 0.5, cluster_info, ha='center', va='center', fontsize=12)
    plt.axis('off')
    
    plt.tight_layout()
    plt.suptitle(f'Discrimination Validation for Cluster {target_cluster}', fontsize=16)
    plt.subplots_adjust(top=0.9)
    
    # Save the summary figure
    plt.savefig(f'cluster_{target_cluster}_validation_summary.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("No high-discrimination clusters identified.")

# 5. Hierarchical Decomposition and Interaction Analysis
Let's analyze which sensitive attributes contribute most to discrimination:

In [ ]:
if high_discrim_clusters:
    print("Hierarchical CMI decomposition for high-discrimination clusters:")
    
    for cluster_id in high_discrim_clusters:
        cluster_data = train_data_with_clusters[train_data_with_clusters['cluster'] == cluster_id]
        
        contributions = hierarchical_cmi_decomposition(
            cluster_data,
            np.zeros(len(cluster_data)),  # All in same cluster
            sensitive_columns,
            outcome_column,
            nonsensitive_columns
        )
        
        print(f"\nCluster {cluster_id} contributions:")
        cluster_cmi = cmi_per_cluster[cluster_id]
        
        # Create a bar chart of contributions
        plt.figure(figsize=(8, 5))
        attrs = list(contributions.keys())
        contrib_values = list(contributions.values())
        percentages = [value/cluster_cmi*100 for value in contrib_values]
        
        plt.bar(attrs, percentages)
        plt.xlabel('Sensitive Attribute')
        plt.ylabel('Contribution (%)')
        plt.title(f'Attribute Contributions to CMI in Cluster {cluster_id}')
        plt.grid(True, alpha=0.3)
        plt.ylim(0, 100)
        
        # Add percentage labels
        for i, p in enumerate(percentages):
            plt.annotate(f'{p:.1f}%', 
                        (i, p), 
                        ha='center', va='bottom')
        
        # Save the figure
        plt.savefig(f'cluster_{cluster_id}_contributions.png', dpi=300, bbox_inches='tight')
        plt.show()  # Keep the show command if you still want to display it
        
        # Print textual results
        for attr, value in contributions.items():
            print(f"  {attr}: {value:.4f} ({value/cluster_cmi*100:.1f}%)")
        
    # Interaction information (if there are at least 2 sensitive attributes)
    if len(sensitive_columns) >= 2:
        print("\nInteraction information for high-discrimination clusters:")
        
        for cluster_id in high_discrim_clusters:
            cluster_data = train_data_with_clusters[train_data_with_clusters['cluster'] == cluster_id]
            
            interaction = interaction_information(
                cluster_data,
                np.zeros(len(cluster_data)),  # All in same cluster
                sensitive_columns[0],
                sensitive_columns[1],
                outcome_column,
                nonsensitive_columns
            )
            
            # Create visualization for interaction information
            plt.figure(figsize=(6, 4))
            plt.bar(['Interaction Information'], [interaction])
            plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
            
            effect_type = "Synergistic" if interaction > 0 else "Redundant"
            plt.title(f'{effect_type} Effect in Cluster {cluster_id}')
            plt.ylabel('Interaction Information')
            plt.annotate(f'{interaction:.4f}', 
                        (0, interaction), 
                        ha='center', 
                        va='bottom' if interaction > 0 else 'top')
            
            # Save the interaction visualization
            plt.savefig(f'cluster_{cluster_id}_interaction.png', dpi=300, bbox_inches='tight')
            plt.show()  # Keep the show command if you still want to display it
            
            print(f"\nCluster {cluster_id}:")
            print(f"  Interaction information: {interaction:.4f}")
            effect_type = "Synergistic" if interaction > 0 else "Redundant"
            print(f"  {effect_type} effect between {sensitive_columns[0]} and {sensitive_columns[1]}")

# 6. Mitigation 
Now, let's apply mitigation strategies to reduce the detected discrimination:

In [ ]:
print("Applying reweighting strategy...")
reweighted_data = reweighting(train_data, best_clusters, sensitive_columns, outcome_column)

# Visualize weight distribution
plt.figure(figsize=(10, 6))
plt.hist(reweighted_data['weight'], bins=30)
plt.xlabel('Weight')
plt.ylabel('Frequency')
plt.title('Distribution of Weights after Reweighting')
plt.grid(True, alpha=0.3)
plt.show()

Let's train a model with fairness regularization:

In [ ]:
# Train model with fairness regularization
print("Training model with fairness regularization...")
fair_model = FairnessRegularizedModel(lambda_param=1.0)
fair_model.fit(X_train, y_train, sensitive_train, best_clusters, nonsensitive_columns)

# Make predictions on test set
y_pred = fair_model.predict(X_test)

# Evaluate accuracy
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {accuracy:.4f}")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Let's apply subgroup-specific calibration:

In [ ]:
# Apply subgroup calibration
print("Applying subgroup-specific calibration...")
# First make sure the outcome column is explicitly identified
outcome_column_name = y_train.name if hasattr(y_train, 'name') else outcome_column

# Make sure train_data has all the necessary columns
train_data_for_calibration = train_data.copy()
if 'cluster' not in train_data_for_calibration.columns:
    train_data_for_calibration['cluster'] = best_clusters

try:
    calibration_results = subgroup_calibration(
        train_data_for_calibration, 
        best_clusters, 
        sensitive_columns,
        # You can optionally provide a base model here
        base_model=None
    )
    
    # Print performances by subgroup
    print("\nCalibration performance by subgroup:")
    
    if 'performance' in calibration_results and calibration_results['performance']:
        performances = calibration_results['performance']
        
        # Sort by improvement
        improvements = [(key, perf['calibrated_acc'] - perf['base_acc']) 
                       for key, perf in performances.items()]
        improvements.sort(key=lambda x: x[1], reverse=True)
        
        for key, improvement in improvements:
            perf = performances[key]
            print(f"  {key}: {perf['base_acc']:.4f} -> {perf['calibrated_acc']:.4f} " + 
                  f"({improvement*100:+.2f}%, {perf['samples']} samples)")
    else:
        print("No performance metrics available. This may be due to insufficient samples in subgroups.")
    
except Exception as e:
    print(f"Error during calibration: {str(e)}")
    print("\nFalling back to simple model comparison without calibration...")
    
    # Train a regular model for comparison
    from sklearn.linear_model import LogisticRegression
    regular_model = LogisticRegression(max_iter=1000)
    regular_model.fit(X_train, y_train)
    
    # Train a weighted model using reweighting
    weighted_data = reweighting(train_data, best_clusters, sensitive_columns, outcome_column_name)
    weighted_model = LogisticRegression(max_iter=1000)
    weighted_model.fit(
        X_train, 
        y_train,
        sample_weight=weighted_data['weight'].values if 'weight' in weighted_data.columns else None
    )
    
    # Compare on test set
    regular_preds = regular_model.predict(X_test)
    weighted_preds = weighted_model.predict(X_test)
    
    from sklearn.metrics import accuracy_score
    print(f"Regular model accuracy: {accuracy_score(y_test, regular_preds):.4f}")
    print(f"Weighted model accuracy: {accuracy_score(y_test, weighted_preds):.4f}")

# 7. Evaluate Mitigation Effectiveness
Finally, let's evaluate how effective our mitigation strategies were at reducing discrimination:


In [ ]:
# Create test data with original and fair predictions
test_data_with_clusters = test_data.copy()

# Get clusters for test data (using nearest centroid)
from sklearn.neighbors import NearestCentroid
centroid_classifier = NearestCentroid()
centroid_classifier.fit(X_train.values, best_clusters)
test_clusters = centroid_classifier.predict(X_test.values)
test_data_with_clusters['cluster'] = test_clusters

# Get predictions from a regular model for comparison
from sklearn.linear_model import LogisticRegression
regular_model = LogisticRegression(max_iter=1000)
regular_model.fit(X_train, y_train)
regular_preds = regular_model.predict(X_test)

# Create datasets with predictions
test_regular = test_data_with_clusters.copy()
test_regular['prediction'] = regular_preds

test_fair = test_data_with_clusters.copy()
test_fair['prediction'] = y_pred

# Calculate CMI for both prediction sets
print("Evaluating mitigation effectiveness...")
regular_cmi = calculate_cmi(
    test_regular,
    test_clusters,
    sensitive_columns,
    'prediction',
    nonsensitive_columns
)

fair_cmi = calculate_cmi(
    test_fair,
    test_clusters,
    sensitive_columns,
    'prediction',
    nonsensitive_columns
)

print(f"CMI on regular predictions: {regular_cmi:.4f}")
print(f"CMI on fair predictions: {fair_cmi:.4f}")
print(f"CMI reduction: {(1 - fair_cmi/regular_cmi)*100:.2f}%")

# Calculate accuracy
regular_acc = accuracy_score(y_test, regular_preds)
fair_acc = accuracy_score(y_test, y_pred)

print(f"Regular model accuracy: {regular_acc:.4f}")
print(f"Fair model accuracy: {fair_acc:.4f}")
print(f"Accuracy change: {(fair_acc - regular_acc)*100:+.2f}%")

# Calculate CMI per cluster
regular_cmi_per_cluster = calculate_cmi_per_cluster(
    test_regular,
    test_clusters,
    sensitive_columns,
    'prediction',
    nonsensitive_columns
)

fair_cmi_per_cluster = calculate_cmi_per_cluster(
    test_fair,
    test_clusters,
    sensitive_columns,
    'prediction',
    nonsensitive_columns
)

# Compare CMI reduction across clusters
print("\nCMI reduction by cluster:")
for cluster in sorted(regular_cmi_per_cluster.keys()):
    if cluster in fair_cmi_per_cluster:
        reg_cmi = regular_cmi_per_cluster[cluster]
        fair_cmi = fair_cmi_per_cluster[cluster]
        reduction = (1 - fair_cmi/reg_cmi)*100
        print(f"  Cluster {cluster}: {reg_cmi:.4f} -> {fair_cmi:.4f} ({reduction:.2f}% reduction)")

# 8. Conclusions
Let's summarize our findings and draw conclusions:

In [ ]:
# Create a summary of our findings
print("Summary of Discrimination Detection Analysis:")
print(f"1. Optimal number of clusters: {optimal_k}")
print(f"2. Best algorithm: {best_algorithm}")
print(f"3. Overall CMI: {cmi_values[best_algorithm]:.4f}")
print(f"4. High discrimination clusters: {high_discrim_clusters}")

if high_discrim_clusters:
    # Get highest contributing attribute for each high-discrimination cluster
    for cluster_id in high_discrim_clusters:
        cluster_data = train_data_with_clusters[train_data_with_clusters['cluster'] == cluster_id]
        
        contributions = hierarchical_cmi_decomposition(
            cluster_data,
            np.zeros(len(cluster_data)),
            sensitive_columns,
            outcome_column,
            nonsensitive_columns
        )
        
        main_contributor = max(contributions.items(), key=lambda x: x[1])[0]
        print(f"5. Cluster {cluster_id} main contributor: {main_contributor}")
        
        if len(sensitive_columns) >= 2:
            interaction = interaction_information(
                cluster_data,
                np.zeros(len(cluster_data)),
                sensitive_columns[0],
                sensitive_columns[1],
                outcome_column,
                nonsensitive_columns
            )
            
            effect_type = "Synergistic" if interaction > 0 else "Redundant"
            print(f"6. {effect_type} effect between {sensitive_columns[0]} and {sensitive_columns[1]}")

print(f"7. Mitigation: {(1 - fair_cmi/regular_cmi)*100:.2f}% CMI reduction with {(fair_acc - regular_acc)*100:+.2f}% accuracy change")

The analysis reveals hidden patterns of discrimination in the Adult dataset that might not be evident from simple aggregate statistics. By using the clustering-based approach with Conditional Mutual Information, we identified specific clusters where discrimination is more pronounced.
Our mitigation strategies successfully reduced this discrimination while maintaining prediction accuracy. The results demonstrate that it's possible to develop fairer automated decision-making systems without compromising performance